Questions.. how do I predict macro trends for Crypto currencies 

### Get the dates and amount of fear greed  

In [256]:
import pandas as pd
savedf = pd.read_csv(r"./Fear Greed Index.csv")

# Dropping rows
savedf = savedf.drop(columns='Unnamed: 1')
savedf = savedf.drop(columns='Unnamed: 2')
savedf = savedf.drop(columns='Unnamed: 3')
savedf = savedf.drop(columns='Unnamed: 4')
savedf = savedf.drop(columns='Unnamed: 5')
savedf = savedf.drop(savedf.tail(5).index)

# Renaming columns 
savedf = savedf.rename(columns={'Unnamed: 0': 'c1'})

### Split the date and the amount into two seporate columns 

In [ ]:
# totally not relavent to the rest of the analysis
# df['c2'] = df.iloc[::2,0]
# df['c3'] = df.iloc[1::2,0]

# df

### Drop NaNs

In [257]:
df = savedf

df = pd.DataFrame(df.c1.values.reshape(-1,2),columns=['c2','c3'])
df = df.rename(columns={'c2': 'date'})
df = df.rename(columns={'c3': 'sentiment'})

savedf = df

### strip the words from C2

In [293]:
df = savedf

df['sentiment'] = df['sentiment'].str[0:2]

savedf = df

,date,sentiment
0,2022-01-24,13
1,2022-01-23,11
2,2022-01-22,13
3,2022-01-21,19
4,2022-01-20,24
...,...,...
1446,2018-02-05,11
1447,2018-02-04,24
1448,2018-02-03,40
1449,2018-02-02,15


#### Add btc price 

In [263]:
btcPrice = pd.read_csv(r"./btcPrice.csv")
btcPrice = btcPrice.drop(columns=['Open', 'High', 'Low', 'Volume', 'Adj Close'])
btcPriceSave = btcPrice

#### Merge BTC prices with dataframe    

In [299]:
btcPrice = btcPriceSave

df = savedf

df.date = pd.to_datetime(df.date)
btcPrice.Date = pd.to_datetime(btcPrice.Date)
btcPrice = btcPrice.rename(columns={'Date': 'date'})
df = pd.merge(btcPrice, df, how='outer', on='date')
df = df.dropna()

savedf = df

In [300]:
df = savedf



,date,Close,sentiment
1233,2018-02-01,9170.540039,30
1234,2018-02-02,8830.750000,15
1235,2018-02-03,9174.910156,40
1236,2018-02-04,8277.009766,24
1237,2018-02-05,6955.270020,11
...,...,...,...
2682,2022-01-20,40680.417969,24
2683,2022-01-21,36457.316406,19
2684,2022-01-22,35030.250000,13
2685,2022-01-23,36276.804688,11
